# Homework 2

**Name:** -- Agustin Villarreal --

**e-mail:** -- agustin.villarreal0743@alumnos.udg.mx --

# MODULES

In [ ]:
# Load modules
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as stats
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from IPython.display import display, HTML


In [6]:
# Define 2Vec class
class Vec2d(object):
    """
    2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
    def __str__(self):
        return f"Vec2d({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Vec2d({self.x}, {self.y})"
    
    def to_tuple(self):
        return (self.x, self.y)

In [7]:
# Module 1: Correlated Random Walk
def generate_crw_trajectory(num_steps, speed, start_pos_x, start_pos_y, cauchy_coef, seed=None):
    """
    Generate a Correlated Random Walk trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient (0 < c < 1)
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Normalize direction and apply speed
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length * speed, direction.y / length * speed)
        
        # Update position
        current_pos = current_pos + direction
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

def plot_crw_trajectory_2d(trajectory, title="Correlated Random Walk Trajectory"):
    """
    Plot a 2D projection of the CRW trajectory using Plotly
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y, time coordinates
    title : str
        Plot title
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add trajectory line
    fig.add_trace(go.Scatter(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        mode='lines',
        name='CRW',
        line=dict(color='red', width=2)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="x_pos (mm)",
        yaxis_title="y_pos (mm)",
        template="plotly_white"
    )
    
    return fig

def plot_crw_trajectory_3d(trajectory, title="Correlated Random Walk Trajectory"):
    """
    Plot a 3D projection of the CRW trajectory using Plotly
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y, time coordinates
    title : str
        Plot title
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add trajectory line
    fig.add_trace(go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=trajectory[:, 2],
        mode='lines',
        name='CRW',
        line=dict(color='red', width=4)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="x_pos",
            yaxis_title="y_pos",
            zaxis_title="time",
        ),
        template="plotly_white"
    )
    
    return fig


In [8]:
# Module 2: Lévy Distribution
def levy_pdf(x, alpha, beta=0, mu=0, c=1):
    """
    Lévy stable distribution PDF approximation
    
    Parameters:
    -----------
    x : array_like
        Points at which to evaluate the PDF
    alpha : float
        Stability parameter (0 < alpha <= 2)
    beta : float, optional
        Skewness parameter (-1 <= beta <= 1)
    mu : float, optional
        Location parameter
    c : float, optional
        Scale parameter
        
    Returns:
    --------
    pdf : array_like
        PDF values at points x
    """
    # Using scipy's implementation of the stable distribution
    return stats.levy_stable.pdf(x, alpha, beta, loc=mu, scale=c)

def plot_levy_distributions(x_range, alphas, mu=3.0, beta=0.0):
    """
    Plot multiple Lévy distributions with different alpha values
    
    Parameters:
    -----------
    x_range : array_like
        Range of x values to plot
    alphas : list
        List of alpha values to plot
    mu : float, optional
        Location parameter
    beta : float, optional
        Skewness parameter
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add a trace for each alpha value
    for alpha in alphas:
        pdf_values = levy_pdf(x_range, alpha, beta, mu)
        
        fig.add_trace(go.Scatter(
            x=x_range,
            y=pdf_values,
            mode='lines',
            name=f'Lévy {alpha}',
            line=dict(width=2)
        ))
    
    # Update layout
    fig.update_layout(
        title="Lévy Stable Distributions",
        xaxis_title="x",
        yaxis_title="Probability Density",
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )
    
    return fig

In [9]:
# Module 3: Lévy Distribution - Histogram + Curve
def generate_levy_samples(n_samples, alpha, beta=0, mu=0, c=1, seed=None):
    """
    Generate samples from a Lévy stable distribution
    
    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    alpha : float
        Stability parameter (0 < alpha <= 2)
    beta : float, optional
        Skewness parameter (-1 <= beta <= 1)
    mu : float, optional
        Location parameter
    c : float, optional
        Scale parameter
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    samples : numpy.ndarray
        Array of samples from the Lévy distribution
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Generate samples using scipy's levy_stable.rvs
    return stats.levy_stable.rvs(alpha, beta, loc=mu, scale=c, size=n_samples)

def plot_levy_histogram_with_curve(samples, alpha, beta=0, mu=0, c=1, bins=50):
    """
    Plot histogram of Lévy samples with the theoretical PDF curve
    
    Parameters:
    -----------
    samples : array_like
        Samples from the Lévy distribution
    alpha : float
        Stability parameter of the samples
    beta : float, optional
        Skewness parameter of the samples
    mu : float, optional
        Location parameter of the samples
    c : float, optional
        Scale parameter of the samples
    bins : int, optional
        Number of histogram bins
        
    Returns:
    --------
    fig : matplotlib.figure.Figure
        Matplotlib figure object
    """
    # Create a range for the PDF
    x_min, x_max = np.min(samples), np.max(samples)
    x_range = np.linspace(x_min, x_max, 1000)
    
    # Calculate the PDF
    pdf_values = levy_pdf(x_range, alpha, beta, mu, c)
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot histogram
    counts, bins, _ = ax.hist(samples, bins=bins, density=True, alpha=0.5, 
                            label='Lévy samples', color='skyblue')
    
    # Plot PDF
    ax.plot(x_range, pdf_values, 'r-', lw=2, label='Lévy PDF')
    
    # Add labels and legend
    ax.set_xlabel('x')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'Lévy Distribution (α={alpha}, β={beta}, μ={mu})')
    ax.legend()
    
    # Improve appearance
    ax.grid(alpha=0.3)
    fig.tight_layout()
    
    return fig


In [10]:
# Module 4: Lévy Flight
def generate_levy_flight_trajectory(num_steps, alpha, beta=0, mu=3.0, c=1.0, 
                                   start_pos_x=0, start_pos_y=0, cauchy_coef=0.7, 
                                   seed=None):
    """
    Generate a Lévy flight trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    alpha : float
        Stability parameter for step length distribution
    beta : float, optional
        Skewness parameter for step length distribution
    mu : float, optional
        Location parameter for step length distribution
    c : float, optional
        Scale parameter for step length distribution
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient for turning angles
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_points, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
    
    # We'll need to generate more steps than requested because
    # Lévy steps will be treated as "number of steps in the same direction"
    max_possible_points = num_steps * 10  # A reasonable upper bound
    
    # Initialize larger trajectory array
    extended_trajectory = np.zeros((max_possible_points, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    extended_trajectory[0, 0] = current_pos.x
    extended_trajectory[0, 1] = current_pos.y
    extended_trajectory[0, 2] = 0  # time
    
    point_index = 1
    step_index = 0
    
    # Generate trajectory
    while step_index < num_steps and point_index < max_possible_points:
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Normalize direction
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length, direction.y / length)
        
        # Sample step length from Lévy distribution
        # We interpret this as "number of steps in the same direction"
        levy_steps = int(max(1, stats.levy_stable.rvs(alpha, beta, loc=mu, scale=c)))
        
        # Take the Lévy number of steps in the current direction
        for i in range(levy_steps):
            if point_index >= max_possible_points:
                break
                
            # Update position (unit step size)
            current_pos = current_pos + direction
            
            # Store position
            extended_trajectory[point_index, 0] = current_pos.x
            extended_trajectory[point_index, 1] = current_pos.y
            extended_trajectory[point_index, 2] = point_index  # time
            
            point_index += 1
        
        step_index += 1
    
    # Trim the trajectory to actual points generated
    trajectory = extended_trajectory[:point_index]
    
    return trajectory

def plot_levy_flight_3d(trajectory, title="Lévy Flight Trajectory"):
    """
    Plot a 3D projection of the Lévy flight trajectory using Plotly
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y, time coordinates
    title : str
        Plot title
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add trajectory line
    fig.add_trace(go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=trajectory[:, 2],
        mode='lines',
        name='Lévy Flight',
        line=dict(color='blue', width=4)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="x_pos (mm)",
            yaxis_title="y_pos (mm)",
            zaxis_title="time",
        ),
        template="plotly_white"
    )
    
    return fig

In [11]:
# Module 5: Multiple CRW Trajectories
def generate_multiple_crw_trajectories(num_trajectories, num_steps, speed, 
                                      start_pos_x, start_pos_y, 
                                      cauchy_coefs, seed=None):
    """
    Generate multiple CRW trajectories with different Cauchy coefficients
    
    Parameters:
    -----------
    num_trajectories : int
        Number of trajectories to generate
    num_steps : int
        Number of steps in each trajectory
    speed : float
        Speed/step size for the walkers
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coefs : list or array
        Cauchy coefficients for each trajectory
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectories : list
        List of trajectory arrays
    """
    if seed is not None:
        np.random.seed(seed)
        
    if len(cauchy_coefs) < num_trajectories:
        raise ValueError("Not enough Cauchy coefficients provided")
    
    trajectories = []
    
    for i in range(num_trajectories):
        traj = generate_crw_trajectory(
            num_steps=num_steps,
            speed=speed,
            start_pos_x=start_pos_x,
            start_pos_y=start_pos_y,
            cauchy_coef=cauchy_coefs[i],
            seed=seed+i if seed is not None else None
        )
        trajectories.append(traj)
    
    return trajectories

def plot_multiple_crw_trajectories_3d(trajectories, cauchy_coefs, 
                                     title="N Correlated Random Walks"):
    """
    Plot multiple CRW trajectories in 3D using Plotly
    
    Parameters:
    -----------
    trajectories : list
        List of trajectory arrays
    cauchy_coefs : list or array
        Cauchy coefficients used for each trajectory
    title : str
        Plot title
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    # Create color scale
    colors = [
        'blue', 'red', 'green', 'purple', 'orange', 
        'cyan', 'magenta', 'yellow', 'pink', 'brown'
    ]
    
    fig = go.Figure()
    
    # Add each trajectory
    for i, (traj, coef) in enumerate(zip(trajectories, cauchy_coefs)):
        color_idx = i % len(colors)
        
        fig.add_trace(go.Scatter3d(
            x=traj[:, 0],
            y=traj[:, 1],
            z=traj[:, 2],
            mode='lines',
            name=f'CRW, Cauchy {coef}',
            line=dict(color=colors[color_idx], width=3)
        ))
    
    # Update layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="x_pos (mm)",
            yaxis_title="y_pos (mm)",
            zaxis_title="time",
        ),
        template="plotly_white"
    )
    
    return fig

In [12]:
# Calculate Mean Squared Displacement (MSD)
def calculate_msd(trajectory):
    """
    Calculate Mean Squared Displacement for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
        
    Returns:
    --------
    tau : numpy.ndarray
        Time lags
    msd : numpy.ndarray
        MSD values for each time lag
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    n_points = len(positions)
    
    # Calculate maximum tau (time lag)
    max_tau = n_points // 4  # Use 1/4 of the trajectory for reliable statistics
    
    tau = np.arange(1, max_tau + 1)
    msd = np.zeros(max_tau)
    
    # Calculate MSD for each tau
    for i, t in enumerate(tau):
        # Calculate squared displacements
        sd = np.sum((positions[t:] - positions[:-t])**2, axis=1)
        # Average to get MSD
        msd[i] = np.mean(sd)
    
    return tau, msd

def plot_msd(tau, msd, title="Mean Squared Displacement"):
    """
    Plot Mean Squared Displacement
    
    Parameters:
    -----------
    tau : array_like
        Time lags
    msd : array_like
        MSD values
    title : str
        Plot title
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add MSD trace
    fig.add_trace(go.Scatter(
        x=tau,
        y=msd,
        mode='lines',
        name='MSD',
        line=dict(color='purple', width=2)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="τ",
        yaxis_title="MSD",
        template="plotly_white"
    )
    
    return fig

In [13]:
# Dashboard UI
def create_random_walks_dashboard():
    """Create an interactive dashboard for random walks visualization"""
    
    # Create tabs for different activities
    tabs = widgets.Tab()
    
    # Activity 1: CRW - 1 Trajectory
    activity1_tab = widgets.VBox()
    
    # Parameters for CRW
    num_steps_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Number of steps')
    speed_slider = widgets.FloatSlider(value=5, min=1, max=20, step=1, description='Speed')
    start_x_slider = widgets.FloatSlider(value=0, min=-100, max=100, step=10, description='Starting pos_x')
    start_y_slider = widgets.FloatSlider(value=0, min=-100, max=100, step=10, description='Starting pos_y')
    cauchy_slider = widgets.FloatSlider(value=0.7, min=0.1, max=0.9, step=0.1, description='Cauchy coefficient')
    
    # Output for plots
    trajectory_output = widgets.Output()
    msd_output = widgets.Output()
    
    # Update button
    update_button = widgets.Button(description='Update Trajectory')
    
    # Function to update plots
    def update_crw_trajectory(_):
        trajectory_output.clear_output()
        msd_output.clear_output()
        
        # Generate trajectory
        trajectory = generate_crw_trajectory(
            num_steps=num_steps_slider.value,
            speed=speed_slider.value,
            start_pos_x=start_x_slider.value,
            start_pos_y=start_y_slider.value,
            cauchy_coef=cauchy_slider.value
        )
        
        # Calculate MSD
        tau, msd = calculate_msd(trajectory)
        
        # Plot trajectory
        with trajectory_output:
            fig = plot_crw_trajectory_3d(trajectory, title="CRW trajectory")
            fig.show()
        
        # Plot MSD
        with msd_output:
            fig = plot_msd(tau, msd, title="Mean Squared Displacement")
            fig.show()
    
    update_button.on_click(update_crw_trajectory)
    
    # Layout for Activity 1
    params_box = widgets.VBox([
        widgets.HTML("<h3>Parameters</h3>"),
        num_steps_slider, 
        speed_slider, 
        start_x_slider, 
        start_y_slider, 
        cauchy_slider,
        update_button
    ])
    
    plots_box = widgets.HBox([
        widgets.VBox([widgets.HTML("<h3>Trajectory</h3>"), trajectory_output]),
        widgets.VBox([widgets.HTML("<h3>Metric</h3>"), msd_output])
    ])
    
    activity1_tab.children = [widgets.HBox([params_box, plots_box])]
    
    # Activity 2: Lévy distribution - N different curves
    activity2_tab = widgets.VBox()
    
    # Parameters for Lévy distribution
    levy_mu_slider = widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='μ parameter')
    levy_beta_slider = widgets.FloatSlider(value=0, min=-1, max=1, step=0.1, description='β parameter')
    
    # Alpha values checkboxes
    alpha_checkboxes = widgets.VBox([
        widgets.Checkbox(value=True, description='α = 0.1'),
        widgets.Checkbox(value=True, description='α = 0.5'),
        widgets.Checkbox(value=True, description='α = 1.0'),
        widgets.Checkbox(value=True, description='α = 1.9')
    ])
    
    # Output for plot
    levy_curves_output = widgets.Output()
    
    # Update button
    update_levy_button = widgets.Button(description='Update Curves')
    
    # Function to update plot
    def update_levy_curves(_):
        levy_curves_output.clear_output()
        
        # Get selected alpha values
        alphas = []
        if alpha_checkboxes.children[0].value:
            alphas.append(0.1)
        if alpha_checkboxes.children[1].value:
            alphas.append(0.5)
        if alpha_checkboxes.children[2].value:
            alphas.append(1.0)
        if alpha_checkboxes.children[3].value:
            alphas.append(1.9)
        
        if not alphas:
            with levy_curves_output:
                print("Please select at least one alpha value.")
            return
        
        # Generate x range
        x_range = np.linspace(-2, 8, 1000)
        
        # Plot curves
        with levy_curves_output:
            fig = plot_levy_distributions(
                x_range=x_range,
                alphas=alphas,
                mu=levy_mu_slider.value,
                beta=levy_beta_slider.value
            )
            fig.show()
    
    update_levy_button.on_click(update_levy_curves)
    
    # Layout for Activity 2
    levy_params_box = widgets.VBox([
        widgets.HTML("<h3>Parameters</h3>"),
        levy_mu_slider,
        levy_beta_slider,
        widgets.HTML("<h4>Alpha Values</h4>"),
        alpha_checkboxes,
        update_levy_button
    ])
    
    levy_plots_box = widgets.VBox([
        widgets.HTML("<h3>Lévy Distributions</h3>"),
        levy_curves_output
    ])
    
    activity2_tab.children = [widgets.HBox([levy_params_box, levy_plots_box])]
    
    # Activity 3: Lévy distribution - Histogram + Curve
    activity3_tab = widgets.VBox()
    
    # Parameters for Lévy histogram
    levy_hist_alpha_slider = widgets.FloatSlider(value=1.5, min=0.1, max=2.0, step=0.1, description='α parameter')
    levy_hist_beta_slider = widgets.FloatSlider(value=0, min=-1, max=1, step=0.1, description='β parameter')
    levy_hist_mu_slider = widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='μ parameter')
    levy_hist_c_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='c parameter')
    levy_hist_samples_slider = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='# Samples')
    levy_hist_bins_slider = widgets.IntSlider(value=50, min=10, max=200, step=10, description='# Bins')
    
    # Output for histogram
    levy_hist_output = widgets.Output()
    
    # Update button
    update_levy_hist_button = widgets.Button(description='Update Histogram')
    
    # Function to update histogram
    def update_levy_histogram(_):
        levy_hist_output.clear_output()
        
        # Generate samples
        samples = generate_levy_samples(
            n_samples=levy_hist_samples_slider.value,
            alpha=levy_hist_alpha_slider.value,
            beta=levy_hist_beta_slider.value,
            mu=levy_hist_mu_slider.value,
            c=levy_hist_c_slider.value
        )
        
        # Plot histogram with curve
        with levy_hist_output:
            fig = plot_levy_histogram_with_curve(
                samples=samples,
                alpha=levy_hist_alpha_slider.value,
                beta=levy_hist_beta_slider.value,
                mu=levy_hist_mu_slider.value,
                c=levy_hist_c_slider.value,
                bins=levy_hist_bins_slider.value
            )
            plt.show()
    
    update_levy_hist_button.on_click(update_levy_histogram)
    
    # Layout for Activity 3
    levy_hist_params_box = widgets.VBox([
        widgets.HTML("<h3>Parameters</h3>"),
        levy_hist_alpha_slider,
        levy_hist_beta_slider,
        levy_hist_mu_slider,
        levy_hist_c_slider,
        levy_hist_samples_slider,
        levy_hist_bins_slider,
        update_levy_hist_button
    ])
    
    levy_hist_plot_box = widgets.VBox([
        widgets.HTML("<h3>Lévy Distribution Histogram</h3>"),
        levy_hist_output
    ])
    
    activity3_tab.children = [widgets.HBox([levy_hist_params_box, levy_hist_plot_box])]
    
    # Activity 4: Lévy flight - Vec2d - 1 Trajectory
    activity4_tab = widgets.VBox()
    
    # Parameters for Lévy flight
    levy_flight_steps_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Number of steps')
    levy_flight_alpha_slider = widgets.FloatSlider(value=1.5, min=0.1, max=2.0, step=0.1, description='α parameter')
    levy_flight_mu_slider = widgets.FloatSlider(value=3.0, min=0, max=10, step=0.5, description='μ parameter')
    levy_flight_c_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='c parameter')
    levy_flight_start_x_slider = widgets.FloatSlider(value=0, min=-100, max=100, step=10, description='Starting pos_x')
    levy_flight_start_y_slider = widgets.FloatSlider(value=0, min=-100, max=100, step=10, description='Starting pos_y')
    levy_flight_cauchy_slider = widgets.FloatSlider(value=0.7, min=0.1, max=0.9, step=0.1, description='Cauchy coefficient')
    
    # Output for Lévy flight trajectory
    levy_flight_output = widgets.Output()
    
    # Update button
    update_levy_flight_button = widgets.Button(description='Update Trajectory')
    
    # Function to update trajectory
    def update_levy_flight(_):
        levy_flight_output.clear_output()
        
        # Generate trajectory
        trajectory = generate_levy_flight_trajectory(
            num_steps=levy_flight_steps_slider.value,
            alpha=levy_flight_alpha_slider.value,
            mu=levy_flight_mu_slider.value,
            c=levy_flight_c_slider.value,
            start_pos_x=levy_flight_start_x_slider.value,
            start_pos_y=levy_flight_start_y_slider.value,
            cauchy_coef=levy_flight_cauchy_slider.value
        )
        
        # Plot trajectory
        with levy_flight_output:
            fig = plot_levy_flight_3d(trajectory, title="Lévy Flight 3D")
            fig.show()
    
    update_levy_flight_button.on_click(update_levy_flight)
    
    # Layout for Activity 4
    levy_flight_params_box = widgets.VBox([
        widgets.HTML("<h3>Parameters</h3>"),
        levy_flight_steps_slider,
        levy_flight_alpha_slider,
        levy_flight_mu_slider,
        levy_flight_c_slider,
        levy_flight_start_x_slider,
        levy_flight_start_y_slider,
        levy_flight_cauchy_slider,
        update_levy_flight_button
    ])
    
    levy_flight_plot_box = widgets.VBox([
        widgets.HTML("<h3>Lévy Flight Trajectory</h3>"),
        levy_flight_output
    ])
    
    activity4_tab.children = [widgets.HBox([levy_flight_params_box, levy_flight_plot_box])]
    
    # Activity 5: Correlated Random Walk - Vec2d - N Trajectories
    activity5_tab = widgets.VBox()
    
    # Parameters for multiple CRW
    multi_crw_n_traj_slider = widgets.IntSlider(value=6, min=2, max=10, step=1, description='Number of trajectories')
    multi_crw_steps_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Steps per trajectory')
    multi_crw_speed_slider = widgets.FloatSlider(value=5, min=1, max=20, step=1, description='Speed')
    multi_crw_start_x_slider = widgets.FloatSlider(value=0, min=-100, max=100, step=10, description='Starting pos_x')
    multi_crw_start_y_slider = widgets.FloatSlider(value=0, min=-100, max=100, step=10, description='Starting pos_y')
    
    # Output for multiple trajectories
    multi_crw_output = widgets.Output()
    
    # Update button
    update_multi_crw_button = widgets.Button(description='Update Trajectories')
    
    # Function to update trajectories
    def update_multi_crw(_):
        multi_crw_output.clear_output()
        
        # Generate Cauchy coefficients
        n_traj = multi_crw_n_traj_slider.value
        cauchy_coefs = np.linspace(0.2, 0.95, n_traj)
        
        # Generate trajectories
        trajectories = generate_multiple_crw_trajectories(
            num_trajectories=n_traj,
            num_steps=multi_crw_steps_slider.value,
            speed=multi_crw_speed_slider.value,
            start_pos_x=multi_crw_start_x_slider.value,
            start_pos_y=multi_crw_start_y_slider.value,
            cauchy_coefs=cauchy_coefs
        )
        
        # Plot trajectories
        with multi_crw_output:
            fig = plot_multiple_crw_trajectories_3d(
                trajectories=trajectories,
                cauchy_coefs=cauchy_coefs,
                title="N Correlated Random Walks"
            )
            fig.show()
    
    update_multi_crw_button.on_click(update_multi_crw)
    
    # Layout for Activity 5
    multi_crw_params_box = widgets.VBox([
        widgets.HTML("<h3>Parameters</h3>"),
        multi_crw_n_traj_slider,
        multi_crw_steps_slider,
        multi_crw_speed_slider,
        multi_crw_start_x_slider,
        multi_crw_start_y_slider,
        update_multi_crw_button
    ])
    
    multi_crw_plot_box = widgets.VBox([
        widgets.HTML("<h3>Multiple CRW Trajectories</h3>"),
        multi_crw_output
    ])
    
    activity5_tab.children = [widgets.HBox([multi_crw_params_box, multi_crw_plot_box])]
    
    # Create tabs and assign children
    tabs.children = [activity1_tab, activity2_tab, activity3_tab, activity4_tab, activity5_tab]
    
    # Set tab titles
    tabs.set_title(0, 'Activity 1: CRW')
    tabs.set_title(1, 'Activity 2: Lévy Curves')
    tabs.set_title(2, 'Activity 3: Lévy Histogram')
    tabs.set_title(3, 'Activity 4: Lévy Flight')
    tabs.set_title(4, 'Activity 5: Multiple CRW')
    
    # Display tabs
    display(widgets.HTML("<h1>Random Walks Dashboard</h1>"))
    display(tabs)
    
    # Initialize plots
    update_crw_trajectory(None)
    update_levy_curves(None)
    update_levy_histogram(None)
    update_levy_flight(None)
    update_multi_crw(None)

In [ ]:
# Main execution
if __name__ == "__main__":
    
    # Create and display the dashboard
    create_random_walks_dashboard()

HTML(value='<h1>Random Walks Dashboard</h1>')